## 허깅페이스 엔드포인트(HuggingFace Endpoints)

__Huggingface Endpoints__

Hugging Face Hub은 12만 개 이상의 모델, 2만 개의 데이터셋, 5만 개의 데모 앱(Spaces)을 보유한 플랫폼으로, 모두 오픈 소스이며 공개적으로 사용 가능합니다. 이 온라인 플랫폼에서 사람들은 쉽게 협업하고 함께 머신러닝을 구축할 수 있습니다.

Hugging Face Hub은 또한 다양한 ML 애플리케이션을 구축하기 위한 다양한 엔드포인트를 제공합니다. 이 예제는 다양한 유형의 엔드포인트에 연결하는 방법을 보여줍니다.

특히, 텍스트 생성 추론은 Text Generation Inference에 의해 구동됩니다. 이는 매우 빠른 텍스트 생성 추론을 위해 맞춤 제작된 Rust, Python, gRPC 서버입니다.

### 허깅페이스 토큰 발급
허깅페이스(https://huggingface.co) 에 회원가입을 한 뒤, 아래의 주소에서 토큰 발급을 신청합니다.

토큰 발급주소: https://huggingface.co/docs/hub/security-tokens

발급받은 토큰을 .env 파일에 HUGGINGFACEHUB_API_TOKEN = 로 지정하거나 

```python 
import os
from dotenv import load_dotenv

load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

```

### HuggingFace 모델 리스트

- 허깅페이스 LLM 리더보드: https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- 모델 리스트: https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads

### Installation and Setup

HuggingFaceEndpoint 클래스를 사용하여 Hugging Face 엔드포인트와 상호 작용할 수 있습니다.

- langchain_community.llms 모듈에서 HuggingFaceEndpoint 클래스를 임포트합니다.
- HuggingFaceEndpoint 클래스를 사용하면 Hugging Face에서 호스팅되는 언어 모델과 통신할 수 있습니다.
- 이 클래스는 Hugging Face 엔드포인트의 URL과 필요한 인증 정보를 사용하여 초기화됩니다.
- 초기화된 HuggingFaceEndpoint 객체를 통해 프롬프트를 전송하고 언어 모델의 응답을 받을 수 있습니다.

In [31]:
# !pip install --upgrade --quiet huggingface_hub

In [1]:
from langchain_community.llms import HuggingFaceEndpoint

In [2]:
from huggingface_hub import login

login()

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

# # 이거 꼭 해야함
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

print(HUGGINGFACEHUB_API_TOKEN[:5])

hf_zw


In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import HuggingFaceEndpoint

template = """존댓말로 답변해주세요. 답변에 대한 이유를 간결하게 답변해주세요. 다음과 같은 형식으로 대답해주세요.
QUESTION: {question}

ANSWER: """

prompt = PromptTemplate.from_template(template)

# 사용할 모델의 저장소 ID를 설정합니다.
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
repo_id = "KT-AI/midm-bitext-S-7B-inst-v1"
# repo_id = "google/gemma-7b"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    top_k=10,           # 상위 K개의 토큰을 선택합니다.
    top_p=0.95,         # 누적 확률이 top_p에 도달할 때까지 토큰을 선택합니다.
    typical_p=0.95,     # typical_p 확률 이상의 토큰만 선택합니다.
    temperature=0.01,  # 샘플링 온도를 설정합니다. 값이 높을수록 더 다양한 출력을 생성합니다.
    repetition_penalty=1.03, # 반복 패널티를 설정합니다. 높을수록 반복을 줄입니다.
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백을 설정합니다.
    streaming=True,  # 스트리밍을 사용합니다.
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
llm_chain = LLMChain(prompt=prompt, llm=llm)
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = llm_chain.invoke(
    {"question": "대한민국 서울에 방문할 때 꼭 가야할 관광지 5곳을 알려줘."}
)

c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\kim_h\.cache\huggingface\token
Login successful


c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


HfHubHTTPError:  (Request ID: thfpPX70haGsxfI-T-0Q5)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/KT-AI/midm-bitext-S-7B-inst-v1.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model KT-AI/midm-bitext-S-7B-inst-v1 is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).